In [ ]:

import numpy as np
def add_white_noise(image, snr_db):

#      # Ensure image is in float format for precise calculations
#     image = image.astype(np.float64)
    # Calculate the power of the original signal (image)
    signal_power = np.mean(image**2)
    
    # Calculate the power of the noise based on the desired SNR (in dB)
    snr_linear = 10**(snr_db / 10)
    # Calculate the noise power based on the desired SNR (in dB)
    noise_power = signal_power / (10**(snr_db / 10))

    
    # Generate noise with the calculated power and zero mean
    noise = np.random.normal(0, np.sqrt(noise_power), image.shape)
    
    # Add the generated noise to the original image
    noisy_image = image + noise
    
    # Clip the values to be in the valid range (0, 255) for image data
    noisy_image = np.clip(noisy_image, 0, 255)

    #  # Convert the image back to unsigned 8-bit format
    # noisy_image = noisy_image.astype(np.uint8)
    
    return noisy_image


In [ ]:
import numpy as np
import cv2
from scipy.signal import convolve2d
import matplotlib.pyplot as plt

# Φορτώνουμε την εικόνα
image = cv2.imread('new_york.png', cv2.IMREAD_GRAYSCALE)

# Ορισμός των παραμέτρων
sigma = 1.7  # Τυπική απόκλιση για το Gauss φίλτρο
SNR = 7     # SNR σε dB

# Δημιουργία του Gauss φίλτρου
def gaussian_kernel(size, sigma):
    k = cv2.getGaussianKernel(size, sigma)
    kernel = np.outer(k, k)
    return kernel

# Εφαρμογή του Gauss φίλτρου στην εικόνα
kernel_size = int(6*sigma + 1)
gaussian_kernel_2d = gaussian_kernel(kernel_size, sigma)
blurred_image = convolve2d(image, gaussian_kernel_2d, mode='same', boundary='wrap')

# Προσθήκη λευκού θορύβου
signal_power = np.mean(blurred_image ** 2)
noise_power = signal_power / (10 ** (SNR / 10))
noise = np.sqrt(noise_power) * np.random.normal(size=image.shape)
noisy_image = blurred_image + noise

# Εμφάνιση των αποτελεσμάτων
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.title('Original Image')
plt.imshow(image, cmap='gray')
plt.axis('off')
plt.subplot(1, 2, 2)
plt.title('Degraded Image')
plt.imshow(noisy_image, cmap='gray')
plt.axis('off')
plt.show()


In [ ]:
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import LogNorm

# Δημιουργία συντεταγμένων X, Y για την κρουστική απόκριση
x = np.linspace(-kernel_size // 2, kernel_size // 2, kernel_size)
y = np.linspace(-kernel_size // 2, kernel_size // 2, kernel_size)
X, Y = np.meshgrid(x, y)

# 3D plot της Κρουστικής Απόκρισης
impulse_response = gaussian_kernel_2d

fig = plt.figure(figsize=(12, 6))
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(X, Y, impulse_response, cmap='hot')

ax.set_title('Κρουστική Απόκριση')
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Απόκριση')
plt.show()

# Δημιουργία συντεταγμένων X, Y για την απόκριση συχνότητας
x_freq = np.fft.fftfreq(image.shape[1])
y_freq = np.fft.fftfreq(image.shape[0])
X_freq, Y_freq = np.meshgrid(x_freq, y_freq)

# Υπολογισμός της απόκρισης συχνότητας
frequency_response = np.fft.fftshift(np.fft.fft2(gaussian_kernel_2d, s=image.shape))
freq_magnitude = np.abs(frequency_response)

# 3D plot της Απόκρισης Συχνότητας
fig = plt.figure(figsize=(12, 6))
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(X_freq, Y_freq, np.log1p(freq_magnitude), cmap='jet')

ax.set_title('Απόκριση Συχνότητας')
ax.set_xlabel('Συχνότητα X')
ax.set_ylabel('Συχνότητα Y')
ax.set_zlabel('Μέγεθος (Log)')
plt.show()

In [ ]:
# Υπολογισμός του Wiener φίλτρου
def wiener_filter(image, kernel, K):
    kernel_ft = np.fft.fft2(kernel, s=image.shape)
    image_ft = np.fft.fft2(image)
    kernel_ft_conj = np.conj(kernel_ft)
    kernel_power_spectrum = np.abs(kernel_ft) ** 2
    wiener_filter_ft = kernel_ft_conj / (kernel_power_spectrum + K)
    restored_image_ft = image_ft * wiener_filter_ft
    restored_image = np.fft.ifft2(restored_image_ft).real
    return restored_image

# Εκτίμηση της ισχύος του θορύβου
K = noise_power / signal_power

# Αποκατάσταση της εικόνας
restored_image = wiener_filter(noisy_image, gaussian_kernel_2d, K)

plt.figure(figsize=(12, 6))
plt.title('Υποβαθμισμένη Εικόνα')
plt.imshow(noisy_image, cmap='gray')
plt.axis('off')
plt.show()

# Εμφάνιση της αποκατεστημένης εικόνας
plt.figure(figsize=(12, 6))
plt.title('Αποκατεστημένη Εικόνα (με γνωστή ισχύ θορύβου)')
plt.imshow(restored_image, cmap='gray')
plt.axis('off')
plt.show()


In [ ]:
# Εκτίμηση της ισχύος του θορύβου από την ενθορυβημένη εικόνα
estimated_noise_power = np.var(noisy_image - blurred_image)
K_estimated = estimated_noise_power / signal_power

# Επαναλαμβανόμενη αποκατάσταση της εικόνας
restored_image_estimated = wiener_filter(noisy_image, gaussian_kernel_2d, K_estimated)

# Εμφάνιση της αποκατεστημένης εικόνας με εκτιμημένη ισχύ θορύβου
plt.figure(figsize=(12, 6))
plt.title('Αποκατεστημένη Εικόνα (με άγνωστη ισχύ θορύβου)')
plt.imshow(restored_image_estimated, cmap='gray')
plt.show()


In [ ]:
# Σύγκριση των εικόνων
plt.figure(figsize=(18, 6))
plt.subplot(1, 3, 1)
plt.title('Υποβαθμισμένη Εικόνα')
plt.imshow(noisy_image, cmap='gray')

plt.subplot(1, 3, 2)
plt.title('Αποκατεστημένη Εικόνα (γνωστή ισχύς θορύβου)')
plt.imshow(restored_image, cmap='gray')

plt.subplot(1, 3, 3)
plt.title('Αποκατεστημένη Εικόνα (εκτιμημένη ισχύς θορύβου)')
plt.imshow(restored_image_estimated, cmap='gray')

plt.show()


